## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-09-14-49-51 +0000,nyt,Bill Gates Makes a Multibillion-Dollar Divorce...,https://www.nytimes.com/2026/01/09/business/de...
1,2026-01-09-14-46-32 +0000,bbc,US job creation in 2025 slows to weakest since...,https://www.bbc.com/news/articles/c0r4zd29n5no...
2,2026-01-09-14-46-00 +0000,wsj,"In a Boost for Free Trade, EU Backs Controvers...",https://www.wsj.com/world/europe/eu-approves-l...
3,2026-01-09-14-44-51 +0000,nyt,What Is the Oreshnik Ballistic Missile Russia ...,https://www.nytimes.com/2026/01/09/world/europ...
4,2026-01-09-14-42-40 +0000,nypost,Italian PM Giorgia Meloni doubts Trump will us...,https://nypost.com/2026/01/09/world-news/melon...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2381/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
34,trump,86
54,venezuela,31
103,ice,29
76,oil,19
104,shooting,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
199,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...,184
243,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...,171
54,2026-01-09-12-23-44 +0000,nyt,Trump Indicates He Will Meet Venezuela’s Macha...,https://www.nytimes.com/2026/01/09/world/ameri...,170
120,2026-01-09-09-00-00 +0000,cbc,How Trump and Vance's accounts of Minneapolis ...,https://www.cbc.ca/news/world/minneapolis-ice-...,163
354,2026-01-08-17-23-16 +0000,nyt,"After ICE Shooting, Trump Administration Deplo...",https://www.nytimes.com/2026/01/08/us/politics...,162


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
199,184,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...
49,87,2026-01-09-12-49-16 +0000,nypost,ICE agent who shot Renee Nicole Good in Minnea...,https://nypost.com/2026/01/09/us-news/jonathan...
243,85,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...
142,64,2026-01-09-05-04-26 +0000,nypost,Russia fires nuclear-capable Oreshnik missile ...,https://nypost.com/2026/01/09/world-news/russi...
155,54,2026-01-09-03-08-27 +0000,nypost,Trump plans to meet Venezuelan opposition lead...,https://nypost.com/2026/01/08/us-news/trump-pl...
310,49,2026-01-08-19-14-24 +0000,nypost,Protests erupt in Iranian capital after exiled...,https://nypost.com/2026/01/08/world-news/prote...
103,47,2026-01-09-10-05-15 +0000,nypost,Iran’s Ayatollah Khamenei previews crackdown o...,https://nypost.com/2026/01/09/world-news/irans...
389,44,2026-01-08-15-00-00 +0000,wsj,The Trump administration’s seizure of tankers ...,https://www.wsj.com/politics/national-security...
193,44,2026-01-09-00-47-00 +0000,wsj,ICE Shooting in Minnesota Raises Rare Legal Qu...,https://www.wsj.com/us-news/law/ice-shooting-i...
214,41,2026-01-08-23-42-00 +0000,wsj,A middle-of-the-night deal back in early July ...,https://www.wsj.com/politics/policy/aca-vote-h...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
